In [ ]:
import numpy as np
import pyvista as pv
from matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cm
import imageio
import os
import gc

# Define Quaternion Julia Set in 3D
def quaternion_julia_3d(grid_size, q_start, q_end, t, max_iter=20):
    """
    Compute the 3D Quaternion Julia set for a specific interpolation of quaternion states.
    """
    q = tuple(q_start[i] * (1 - t) + q_end[i] * t for i in range(4))
    x = np.linspace(-1.5, 1.5, grid_size)
    y = np.linspace(-1.5, 1.5, grid_size)
    z = np.linspace(-1.5, 1.5, grid_size)
    X, Y, Z = np.meshgrid(x, y, z, indexing="ij")
    W = X + Y * 1j + Z * 1j**2
    iteration_counts = np.zeros(W.shape, dtype=int)

    for i in range(max_iter):
        mask = np.abs(W) <= 2
        iteration_counts[mask] += 1
        W[mask] = W[mask] ** 2 + (q[0] + q[1] * 1j + q[2] * 1j**2 + q[3] * 1j**3)

    return iteration_counts

# Select colormap
def select_colormap(color_scheme):
    if color_scheme == "ultraviolet":
        return LinearSegmentedColormap.from_list("ultraviolet", ["#000000", "#8A2BE2", "#9400D3", "#FF00FF", "#FFFFFF"])
    elif color_scheme == "neon":
        return LinearSegmentedColormap.from_list("neon", ["#000000", "#00FFFF", "#00FF00", "#FFFF00", "#FFFFFF"])
    elif color_scheme == "chartreuse":
        return LinearSegmentedColormap.from_list("chartreuse", ["#000000", "#7FFF00", "#ADFF2F", "#FFFFFF"])
    elif color_scheme == "fiery_sunset":
        return LinearSegmentedColormap.from_list("fiery_sunset", ["#000000", "#FF4500", "#FF6347", "#FFD700", "#FFFFFF"])
    elif color_scheme == "deep_ocean":
        return LinearSegmentedColormap.from_list("deep_ocean", ["#000000", "#00008B", "#1E90FF", "#87CEEB", "#FFFFFF"])
    else:
        raise ValueError(f"Unknown color scheme: {color_scheme}")

# Render a 3D volume with a spinning camera
def render_volume(data, color_scheme, filename=None, camera_angle=0):
    """
    Render the 3D Quaternion Julia set using pyvista with a spinning camera.
    
    Parameters:
        data (numpy.ndarray): 3D array of iteration counts.
        color_scheme (str): Color scheme name.
        filename (str): Optional filename to save the frame.
        camera_angle (float): The angle (in degrees) to rotate the camera around the object.
    """
    normalized_data = data / np.max(data)
    cmap = select_colormap(color_scheme)
    pyvista_cmap = cm.get_cmap(cmap)

    x = np.linspace(-1.5, 1.5, data.shape[0])
    y = np.linspace(-1.5, 1.5, data.shape[1])
    z = np.linspace(-1.5, 1.5, data.shape[2])
    X, Y, Z = np.meshgrid(x, y, z, indexing="ij")
    points = np.c_[X.ravel(), Y.ravel(), Z.ravel()]
    grid = pv.StructuredGrid()
    grid.points = points
    grid.dimensions = data.shape
    grid["values"] = data.ravel()

    plotter = pv.Plotter(off_screen=True)
    plotter.add_volume(grid, scalars="values", cmap=pyvista_cmap, opacity="sigmoid_6")
    plotter.set_background("black")
    plotter.show_axes = False

    # Adjust camera position to spin around the object
    plotter.camera_position = [
        (np.sin(np.radians(camera_angle)) * 5, np.cos(np.radians(camera_angle)) * 5, 3),  # Position
        (0, 0, 0),  # Focal point (center of the object)
        (0, 0, 1),  # View-up direction
    ]

    if filename:
        plotter.show(screenshot=filename)
    plotter.close()

# Generate frames for a GIF
def generate_gif_frames(grid_size, quaternions, num_frames, color_scheme, max_iter=20):
    """
    Generate frames for a GIF that transitions between multiple quaternion states with a spinning camera.
    """
    frames = []
    num_states = len(quaternions)
    frames_per_transition = num_frames // (num_states - 1)
    total_frames = frames_per_transition * (num_states - 1)

    for i in range(num_states - 1):
        q_start, q_end = quaternions[i], quaternions[i + 1]
        for j in range(frames_per_transition):
            t = j / frames_per_transition
            camera_angle = (360 * 2) * ((i * frames_per_transition + j) / total_frames)  # Spin the camera twice
            print(f"Generating frame {i * frames_per_transition + j + 1}/{num_frames}...")
            data = quaternion_julia_3d(grid_size, q_start, q_end, t, max_iter)
            filename = f"frame_{i * frames_per_transition + j}.png"
            render_volume(data, color_scheme, filename, camera_angle)
            frames.append(imageio.imread(filename))
            os.remove(filename)  # Clean up
            gc.collect()  # Clear memory
    return frames

# Create GIF
def create_gif(frames, output_filename, frame_duration=0.05):
    imageio.mimsave(output_filename, frames, duration=frame_duration)

# Main function
def main():
    grid_size = 50  # Resolution of the 3D grid
    num_frames = 60  # Number of frames in the animation
    quaternions = [
        (0.355, 0.355, 0.355, 0.355),  # Start state
        (0.4, -0.4, -0.4, 0.4),
        (0.1, 0.5, -0.2, 0.8),
        (-0.5, 0.2, -0.1, -0.4),
        (0.2, 0.6, 0.4, 0.2)  # Final state
    ]
    color_schemes = ["ultraviolet", "neon", "chartreuse", "fiery_sunset", "deep_ocean"]
    output_folder = "./output_gifs_3d"
    os.makedirs(output_folder, exist_ok=True)

    for color_scheme in color_schemes:
        print(f"Generating GIF for color scheme: {color_scheme}")
        frames = generate_gif_frames(grid_size, quaternions, num_frames, color_scheme)
        output_filename = os.path.join(output_folder, f"quaternion_julia_3d_{color_scheme}.gif")
        create_gif(frames, output_filename, frame_duration=0.05)
        print(f"Saved GIF: {output_filename}")

# Run the program
if __name__ == "__main__":
    main()